# Non Adaptive Importance Sampling algorithm with openturns

Source of the algorithm : J. Morio & M. Balesdent, Estimation of Rare Event Probabilities in Complex Aerospace and Other Systems, A Practical Approach, Elsevier, 2015


The theory is given for a failure event defined as $\phi(\mathbf{X})>S$ with $\mathbf{X}$ a random vector following a joint PDF $h_0$, $S$ a threshold and $\phi$ a limit state function, without loss of generality.

The IS probability estimate by Importance Sampling $\widehat{P}^{IS}$ is given by 
\begin{equation}
\widehat{P}^{IS}=\frac{1}{N} \sum_{i=1}^{N} {\bf 1}_{\phi(\mathbf{X}_i)>S} \frac{h_0(\mathbf{X}_i)}{h(\mathbf{X}_i)}.
\label{ISeq}
\end{equation}

It is well-known that the optimal density minimizing the variance of the estimator $h_{opt}$ is  defined as
\begin{equation}
h_{opt}=\frac{{\mathbf 1}_{\phi(x)>S}h_0}{P}.
\label{opt}
\end{equation}


with $P$ the failure probability and is inaccessible in practice since this probability is unknown. 


The objective of Non parametric Adaptive Importance Sampling (NAIS) technique is to approximate the IS optimal auxiliary density given in the preceding equation  with a kernel function (e.g. Gaussian kernel). NAIS does not require the choice of a parametric pdf family  as Cross Entropy and is thus more flexible than a parametric model. Its iterative principle is relatively similar to CE optimization and can be described by the following steps.


1. $k=1$ and set $\rho \in [0,1]$
2. Generate the population $\mathbf{X}_1^{(k)},\dots,\mathbf{X}_N^{(k)}$ according to the pdf $h_{k-1}$, apply the function $\phi$ in order to have $Y_1^{(k)}=\phi(\mathbf{X}_1^{(k)}),\ldots,Y_N^{(k)}=\phi(\mathbf{X}_N^{(k)})$
3. Compute the empirical $\rho$-quantile $q_k=\min(S, Y^{(k)}_{\left\lfloor\rho N\right\rfloor})$
4. Estimate $I_k= \frac{1}{kN} \displaystyle \sum_{j=1}^{k}\sum_{i=1}^{N} {\bf 1}_{\phi(\mathbf{X}_i^{(j)}) \geq q_k} \frac{h_0(\mathbf{X}_i^{(j)})}{h_{j-1}(\mathbf{X}_i^{(j)})}$ 
5. Update the Gaussian kernel sampling pdf with
\begin{equation}
h_{k}(\mathbf{X})=\frac{1}{k N I_k \det\left(B_{k+1}\right)}\sum_{j=1}^{k}\sum_{i=1}^{N}  w_{j}(\mathbf{X}_i^{(j)})K_d\left(B_{k+1}^{-1}\left(\mathbf{X}-\mathbf{X}_i^{(j)}\right)\right),
\end{equation}
where $K_d$ is the standard $d$-dimensional Gaussian function with zero mean and a diagonal covariance matrix $B_{k+1}=diag(b^1_{k+1},...,b^d_{k+1})$ and $w_j={\bf 1}_{\phi(\mathbf{X}_i^{(j)}) \geq q_k} \frac{h_0(\mathbf{X}_i^{(j)})}{h_{j-1}(\mathbf{X}_i^{(j)})}$. The coefficients of matrix $B_{k+1}$ can be approximated (Silverman Rule) or postulated according to the AMISE (Asymptotic Mean Integrated Square Error) criterion for example. 
6. If $q_k<S$, $k\leftarrow k+1$, go to Step 2
7. Estimate the probability $\widehat{P}^{NAIS}(\phi(\mathbf{\mathbf{X}}>S))=\frac{1}{N}\displaystyle \sum_{i=1}^{N} 1_{\phi(\mathbf{X}_i^{(k)})>S} \frac{h_0(\mathbf{X}_i^{(k)})}{h_{k-1}(\mathbf{X}_i^{(k)})}$


The NAIS algorithm with the Silverman rule is coded in the following class.


In [ ]:
import numpy as np
import openturns as ot
from NAISAlgorithm import NAISAlgorithm 

## Numerical experiments : Four branch function

$$G(x_1,x_2) = min \begin{pmatrix}3+0.1(x_1-x_2)^2-\frac{(x_1+x_2)}{\sqrt{2}};\\3+0.1(x_1-x_2)^2+\frac{(x_1+x_2)}{\sqrt{2}};\\
(x_1-x_2)+ \frac{k}{\sqrt{2}};\\
(x_2-x_1)+ \frac{k}{\sqrt{2}}
\end{pmatrix}$$

with : 
* $k$ is equal to 6 or 7
* $x_1 \sim \mathcal{N}(0,1)$
* $x_2 \sim \mathcal{N}(0,1)$


In [ ]:
#Definition of limit state function
def four_branch(x):
    x1 = x[0]
    x2  = x[1]
    k = x[2]
    
    g1 = 3+0.1*(x1-x2)**2-(x1+x2)/np.sqrt(2)
    g2 = 3+0.1*(x1-x2)**2+(x1+x2)/np.sqrt(2)
    g3 = (x1-x2)+k/np.sqrt(2)
    g4 =(x2-x1)+k/np.sqrt(2)
    
    return [min((g1,g2,g3,g4))]


# Definition de la pythonfunction generale
my_four_branch = ot.PythonFunction(3, 1, four_branch)

# Transformation de la pythonfunction vers parametricfunction en figeant le parametre k 
index_frozen = [2]
my_four_branch_6 = ot.ParametricFunction(my_four_branch, index_frozen, [6])
my_four_branch_7 = ot.ParametricFunction(my_four_branch, index_frozen, [7])

In [ ]:
#Definition of input variable PDF

dim_inputs = 2
dist_x = ot.Normal([0.0, 0.0], [1.0, 1.0], ot.CorrelationMatrix(dim_inputs))
inputVector = ot.RandomVector(dist_x)

In [ ]:
#Determination of reference probability
#MonteCarlo experiment

n_MC = np.int(1e6)

# Creation of event
ot.RandomGenerator.SetSeed(1)

vect = ot.RandomVector(dist_x)
G = ot.CompositeRandomVector(my_four_branch_7, vect)
event = ot.ThresholdEvent(G, ot.Less(), 0.0)

# create a Monte Carlo algorithm
experiment = ot.MonteCarloExperiment()
algo = ot.ProbabilitySimulationAlgorithm(event, experiment)
algo.setMaximumOuterSampling(int(n_MC))
algo.run()

# retrieve results
result = algo.getResult()
probability = result.getProbabilityEstimate()
print('Pf=', probability)

In [ ]:
# Hyperparameters of the algorithm
n_IS= 1000 # Number of samples at each iteration
rho_quantile= 25 # Quantile determining the percentage of failure samples in the current population 

# Definition of the algoritm
NAIS = NAISAlgorithm(event,n_IS,rho_quantile)

# Run of the algorithm
NAIS.run()
NAIS_result = NAIS.getResult()

print('Probability of failure:',NAIS_result.getProbabilityEstimate())
print('Samples:',NAIS_result.getSamples())


In [ ]:
##plot 
import numpy as np
#Plot of surrogate model
import matplotlib.pyplot as plt
import matplotlib as mpl
grid_size = 100
x1 = np.linspace(-4,4,grid_size)
x2 = np.linspace(-4,4,grid_size)

xx1,xx2 = np.meshgrid(x1,x2)

xx1_ = xx1.reshape((grid_size**2,1))
xx2_ = xx2.reshape((grid_size**2,1))

x = np.concatenate((xx1_,xx2_),1)
                  
y_true = np.array(my_four_branch_7(x))

y_true = y_true.reshape((grid_size,grid_size))

cmap = mpl.cm.hsv
norm = mpl.colors.Normalize(vmin=-10, vmax=10)
%matplotlib inline 
fig, (ax0) = plt.subplots(ncols=1,figsize=(9,9))
im1 = ax0.pcolormesh(xx1,xx2,y_true,norm = norm,cmap = cmap)
fig.colorbar(im1, ax=ax0)
ax0.title.set_text('True function')

In [ ]:
#Plot of auxiliary density 

density_4_branch = NAIS_result.getAuxiliaryDensity()
density_4_branch.drawPDF()

In [ ]:
#Plot of samples on function 

import numpy as np
#Plot of surrogate model
import matplotlib.pyplot as plt
import matplotlib as mpl
grid_size = 100
x1 = np.linspace(-4,4,grid_size)
x2 = np.linspace(-4,4,grid_size)

xx1,xx2 = np.meshgrid(x1,x2)

xx1_ = xx1.reshape((grid_size**2,1))
xx2_ = xx2.reshape((grid_size**2,1))

x = np.concatenate((xx1_,xx2_),1)
                  
y_true = np.array(my_four_branch_7(x))

y_true = y_true.reshape((grid_size,grid_size))

cmap = mpl.cm.hsv
norm = mpl.colors.Normalize(vmin=-10, vmax=10)
%matplotlib inline 
fig, (ax0) = plt.subplots(ncols=1,figsize=(9,9))
im1 = ax0.pcolormesh(xx1,xx2,y_true,norm = norm,cmap = cmap)
fig.colorbar(im1, ax=ax0)
ax0.title.set_text('True function')

Samples = NAIS_result.getSamples()
plt.scatter(Samples[:,0],Samples[:,1],linewidths = 0.01,marker='+',color='r')
ax = plt.gca()
ax.set_xlim([-4,4])
ax.set_ylim([-4,4])